In [5]:
entropy_loss='cross_entropy_loss'
squared_loss='mean_squared_error'

fmnist_dataset='fashion_mnist'
mnist_dataset='mnist'



In [6]:
from keras.datasets import fashion_mnist,mnist
import wandb
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import copy
import argparse
from types import SimpleNamespace
import wandb

class Activation_Functions:
    def __init__(self,function):
        self.function=function
    def getActivations(self,x):
        if self.function=='relu':
            return np.maximum(0,x)
        elif self.function=='sigmoid':
            return 1.0/(1+np.exp(-x))
            return sigmoid(x)
        elif self.function=='tanh':
            return np.tanh(x)
    def getDerivatives(self,x):
        if self.function=='relu':
             return 1*(x>0)
        elif self.function=='sigmoid':
            return sigmoid(x)*(1-sigmoid(x))
        elif self.function=='tanh':
            return (1 - (np.tanh(x)**2))         

def relu(x):
    return np.maximum(0,x)

def reluPrime(x):
    return 1*(x>0)

class Optimizers:
    def __init__(self,method):
        self.method=method
    def update(self,obj,x_batch=None,y_batch=None):
        
        lr=obj.params.learning_rate
        momentum=0.9
        adam_beta1=0.9
        adam_beta2=0.99
        rms_beta=0.9
        weight_decay=obj.params.weight_decay
        epsilon=1e-9
        

        if self.method=='sgd':
            for i in range(len(obj.layers)):
                obj.weights[i]=obj.weights[i]-lr*(2*weight_decay*obj.weights[i]+obj.gradients_w[i])
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        elif self.method=='momentum':
            init=0
            new_gradients_w=[None]*len(obj.gradients_w)
            new_gradients_b=[None]*len(obj.gradients_b)
            for i in range(len(obj.layers)):
                if obj.prev_gradients_w!=None:
                    new_gradients_w[i]=(1-momentum)*obj.gradients_w[i]+obj.prev_gradients_w[i]*momentum
                    new_gradients_b[i]=(1-momentum)*obj.gradients_b[i]+obj.prev_gradients_b[i]*momentum
                    
                else:
                    new_gradients_w[i]=lr*obj.gradients_w[i]
                    new_gradients_b[i]=lr*obj.gradients_b[i]
                    
                obj.weights[i]=obj.weights[i]-lr*(new_gradients_w[i]+2*weight_decay*obj.weights[i])
                obj.biases[i]=obj.biases[i]-lr*(new_gradients_b[i])
                
            obj.prev_gradients_w=copy.deepcopy(new_gradients_w)
            obj.prev_gradients_b=copy.deepcopy(new_gradients_b)
            
            
            
            
            
            pass
        elif self.method=='rmsprop':
            for i in range(len(obj.layers)):
                obj.rms_v[i]=rms_beta*obj.rms_v[i]+(1-rms_beta)*(obj.gradients_w[i]**2)
                
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(obj.rms_v[i])+epsilon))*(obj.gradients_w[i])
                 
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i] 
                
        elif self.method=='adam':
            for i in range(len(obj.layers)):
                obj.adam_m[i]=adam_beta1*obj.adam_m[i]+(1-adam_beta1)*obj.gradients_w[i]
                m_hat=obj.adam_m[i]/(1-np.power(adam_beta1,obj.counter))
                
                obj.adam_v[i]=adam_beta2*obj.adam_v[i]+(1-adam_beta2)*obj.gradients_w[i]*obj.gradients_w[i]
                v_hat=obj.adam_v[i]/(1-np.power(adam_beta2,obj.counter))
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(v_hat)+epsilon))*m_hat
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        
        elif self.method=='nadam':
             for i in range(len(obj.layers)):
                obj.adam_m[i]=adam_beta1*obj.adam_m[i]+(1-adam_beta1)*obj.gradients_w[i]
                m_hat=obj.adam_m[i]/(1-adam_beta1)
                
                obj.adam_v[i]=adam_beta2*obj.adam_v[i]+(1-adam_beta2)*obj.gradients_w[i]*obj.gradients_w[i]
                v_hat=obj.adam_v[i]/(1-adam_beta2)
                obj.weights[i]=obj.weights[i]-(lr/(np.sqrt(v_hat)+epsilon))*m_hat
                
                obj.weights[i]=obj.weights[i]-(lr*(np.divide((adam_beta1*m_hat)+((1-adam_beta1)/(1-np.power(adam_beta1,obj.counter)))*obj.gradients_w[i],np.sqrt(v_hat)+1e-9)))
                obj.biases[i]=obj.biases[i]-lr*obj.gradients_b[i]
        elif self.method=='nag':
            original_weights=copy.deepcopy(obj.weights)
            original_biases=copy.deepcopy(obj.biases)
            original_graidents_w=copy.deepcopy(obj.gradients_w)
            original_graidents_b=copy.deepcopy(obj.gradients_b)
            
            if obj.prev_gradients_w!=None:
                 for i in range(len(obj.layers)):
                        obj.weights[i]=obj.weights[i]-momentum*obj.gradients_w[i]
                        obj.biases[i]=obj.biases[i]-momentum*obj.gradients_b[i]
            
            layer_outs,inter_values=obj.forward(x_batch)
            loss=0
            if(obj.loss_function==entropy_loss):
                loss=cross_entropy_loss(layer_outs[-1],y_batch)
            elif(obj.loss_function==squared_loss):
                loss=cross_entropy_loss(layer_outs[-1],y_batch)
            obj.compute_deltas(layer_outs,inter_values,y_batch)
            obj.find_gradients(x_batch,layer_outs)
            
            for i in range(len(obj.gradients_w)):
                if obj.prev_gradients_w!=None:
                    obj.gradients_w[i]=original_graidents_w[i]*momentum+obj.gradients_w[i]
                    obj.gradients_b[i]=original_graidents_b[i]*momentum+obj.gradients_b[i]
                    
                obj.weights[i]=original_weights[i]-lr*(obj.gradients_w[i]+2*weight_decay*obj.weights[i])
                obj.biases[i]=original_biases[i]-lr*(obj.gradients_b[i])
            obj.prev_gradients_w=True
            return loss




def sigmoid(x):
    x=np.clip(x,-500,500)
    return 1.0/(1.0+1.0*np.exp(-x))

    
def sigmoidPrime(x):
    return sigmoid(x)*(1-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanhPrime(x):
    return (1 - (np.tanh(x)**2))

def intialize_weights(method,rows,cols):
    if method=='uniform':
        return np.random.rand(rows,cols)
    elif method=='xavier':
        return np.random.randn(rows,cols)*np.sqrt(2/(rows+cols))
    else:
        return np.random.randn(rows,cols)

def soft_max(x):
    max_=np.max(x,axis=0)
    x=x-max_
    return np.exp(x)/np.sum(np.exp(x),axis=0)

def soft_max_prime(x):
    return soft_max(x)*(1-soft_max(x))
    

class OneHotEncoder:
    def __init__(self,max_):
        self.max_=max_
    def transform(self,x):
        out=np.zeros((len(x),self.max_))
        out[np.arange(len(x)),x]=1
        return out.T

def cross_entropy_loss(y_pre,y):
    y_pre[y_pre<1e-15]=1e-15
    loss=-np.sum(y*np.log(y_pre))
    return loss/float(y_pre.shape[1])

def mean_squared_loss(y_pre,y):
    return np.mean((y_pre-y)**2)

def getSig():
    return lambda x:1.0/(1+np.exp(-x))

def pre_process(x):
    x=x.reshape(-1,784)
    x=x/255
    return x

class NN:
    def __init__(self,input_size,layers,params):
        self.input_size=input_size
        self.layers=layers
        self.nlayers=len(layers)
        self.activations=Activation_Functions(params.activation)
        self.optimizer=Optimizers(params.optimizer)
        self.weights=[]
        self.weights_init=params.weight_init
        self.weights.append(intialize_weights(self.weights_init,layers[0],input_size))
        self.deltas=[None]*self.nlayers
        self.gradients_w=[None]*self.nlayers
        self.gradients_b=[None]*self.nlayers
        self.prev_gradients_w=None
        self.pre_gradients_b=None
        self.loss_function=params.loss
        self.counter=1
        self.biases=[]
        self.batch_size=params.batch_size
        for i in range(1,len(layers)):
            self.weights.append(intialize_weights(self.weights_init,layers[i],layers[i-1]))
        for i in range(len(layers)):
            self.biases.append(np.random.randn(layers[i],1))   
        
        if self.optimizer.method in ['adam','nadam']:
            self.adam_m=copy.deepcopy(self.weights)
            self.adam_v=copy.deepcopy(self.adam_m)
            _=[each.fill(0) for each in self.adam_m]
            _=[each.fill(0) for each in self.adam_v]
        if self.optimizer.method=='rmsprop':
            self.rms_v=copy.deepcopy(self.weights)
            _=[each.fill(0) for each in self.rms_v]
        self.params=params
        
        
    def forward(self,x):
        layer_outputs=[]
        inter_values=[]
        
        for i in range(len(self.weights)):
            inter_values.append(self.weights[i].dot(x)+self.biases[i])
            if i!= len(self.weights)-1:
                layer_outputs.append(self.activations.getActivations(inter_values[-1]))
                x=layer_outputs[-1]
        layer_outputs.append(soft_max(inter_values[-1]))
        return layer_outputs,inter_values
    def compute_deltas(self,layer_outputs,inter_values,y_one_hot):
        if(self.loss_function==entropy_loss):
            self.deltas[self.nlayers-1]=(layer_outputs[self.nlayers-1]-y_one_hot)
        elif(self.loss_function==squared_loss):
            self.deltas[self.nlayers-1] = (layer_outputs[self.nlayers-1]-y_one_hot) * soft_max_prime(inter_values[self.nlayers-1])
            
        for i in range(self.nlayers-2,-1,-1):
            self.deltas[i]=np.matmul(self.weights[i+1].T,self.deltas[i+1])*self.activations.getDerivatives(inter_values[i])
        
    def find_gradients(self,x,layer_outs):
        self.gradients_w[0]=(np.matmul(self.deltas[0],x.T)+2*self.params.weight_decay*self.weights[0])/self.batch_size
        for i in range(1,self.nlayers):
            self.gradients_w[i]=(np.dot(self.deltas[i],layer_outs[i-1].T)+2*self.params.weight_decay*self.weights[i])/self.batch_size
        for i in range(self.nlayers):
            self.gradients_b[i]=np.sum(self.deltas[i],keepdims=True)/self.batch_size
             
    def update_weights(self,lr):
        for each in range(self.nlayers):
            self.optimizer.update(self)
    
            
    def train(self,x,y,x_valid,y_valid):
        lr=self.params.learning_rate
        batch_size=self.params.batch_size
        train_losses=[]
        valid_losses=[] 
        epochs=self.params.epochs
        for epoch in range(epochs):                
            i=0
            batch_count=0
            loss=0.0
            while i+batch_size < x.shape[1]:
                batch_count+=1
                x_batch=x[:,i:i+batch_size]
                y_batch=y[:,i:i+batch_size]
                i+=batch_size
                if self.optimizer.method!='nag':
                    layer_outs,inter_values=self.forward(x_batch)
                    if(self.loss_function==entropy_loss):
                        loss+=cross_entropy_loss(layer_outs[-1],y_batch)
                    elif(self.loss_function==squared_loss):
                        loss+=mean_squared_loss(layer_outs[-1],y_batch)
                    self.compute_deltas(layer_outs,inter_values,y_batch)
                    self.find_gradients(x_batch,layer_outs)
                    self.optimizer.update(self)
                else:
                    loss+=self.optimizer.update(self,x_batch=x_batch,y_batch=y_batch)
                self.counter+=1
                    
            train_losses.append(loss/batch_count)
            layer_outs,inter_values=self.forward(x_valid)
            if(self.loss_function==entropy_loss):
                valid_loss=cross_entropy_loss(layer_outs[-1],y_valid)
            else:
                valid_loss=mean_squared_loss(layer_outs[-1],y_valid)
                
            valid_losses.append(valid_loss)
            tr_ac=self.calc_accuracy(x,y)
            val_ac=self.calc_accuracy(x_valid,y_valid)
            print(f"epoch {epoch+1} : train loss = {train_losses[-1]} valid loss = {valid_loss}")
            wandb.log({'train loss':train_losses[-1],'train accuracy':tr_ac,'valid loss':valid_loss,'valid accuracy':val_ac})
        return train_losses,tr_ac,valid_losses,val_ac
    
    def predict_probas(self,x):
        layer_outs,inter_values=self.forward(x)
        return layer_outs[-1]
        
    def calc_accuracy(self,x,y):
        layer_outs,inter_values=self.forward(x)
        pred=np.argmax(layer_outs[-1],axis=0)
        expected=np.argmax(y,axis=0)
        
        return (np.sum(pred==expected)/y.shape[1])*100
               

def load_data(dataset=fmnist_dataset):
    if dataset==fmnist_dataset:
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    elif dataset==mnist_dataset:
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    split_size=0.8

    x_train,x_valid=x_train[:int(len(x_train)*split_size)],x_train[int(len(x_train)*split_size):] #splitting train into train and valid
    y_train,y_valid=y_train[:int(len(y_train)*split_size)],y_train[int(len(y_train)*split_size):]

    x_train=pre_process(x_train)
    x_valid=pre_process(x_valid)
    x_test=pre_process(x_test) 
    
    one_hot=OneHotEncoder(10)
    y_train=one_hot.transform(y_train)
    y_valid=one_hot.transform(y_valid)
    y_test=one_hot.transform(y_test)
    
    
    return x_train,y_train,x_valid,y_valid,x_test,y_test

x_train,y_train,x_valid,y_valid,x_test,y_test=load_data(dataset=fmnist_dataset)
    

In [7]:
sweep_config = {
    'method': 'random', #grid, random,#bayes
    'name' : 'Random_sweep_cross_entropy',
    'metric': {
      'name': 'valid accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [10,15]
        },
        'num_layers':{
            'values':[3,4,5]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'weight_decay':{
            'values':[0,0.0005,0.5]
        },
         'learning_rate':{
            'values':[0.1,0.01,0.001]
        },
         'optimizer': {
            'values': ['sgd','momentum','nag','rmsprop','adam','nag']
        },
        'batch_size': {
            'values': [16,32,64]
        },
         'weight_init':{
            'values':['normal','random','xavier']
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        }, 
        'loss': {
            'values': [entropy_loss]
        }, 
        
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_Assign_1')

Create sweep with ID: 2z1sl0ve
Sweep URL: https://wandb.ai/cs22m080/DL_Assign_1/sweeps/2z1sl0ve


In [ ]:
def main():
    with wandb.init() as run:
        params={}
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        
        layers=[params.hidden_size]*params.num_layers
        run_name="opt_"+wandb.config.optimizer+"-ac_"+wandb.config.activation+"-batch_"+str(wandb.config.batch_size)\
                + "-nlayers_"+str(wandb.config.num_layers)+"-epochs_"+str(wandb.config.epochs)
        wandb.run.name=run_name
        layers.append(10)
        obj=NN(784,layers,params)
        train_losses,tr_ac,valid_losses,val_ac=obj.train(x_train.T,y_train,x_valid.T,y_valid)
    
wandb.agent(sweep_id, function=main,count=100)

wandb: Agent Starting Run: jnzzjn2r with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	loss: cross_entropy_loss
wandb: 	num_layers: 4
wandb: 	optimizer: nag
wandb: 	weight_decay: 0
wandb: 	weight_init: normal
